In [87]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [88]:
df = pd.read_csv('/content/penguins_size.csv')

df = df.dropna()
df = df.drop(336, axis = 0)

X = df.drop('body_mass_g', axis=1)
y = df['body_mass_g']


class_name = ["culmen_length_mm", "culmen_depth_mm", "flipper_length_mm"]
class_name2 = ["species","island", "sex"]


num_pipeline = Pipeline([
    ("scale", StandardScaler())
])

cat_pipeline = Pipeline([
    ("encode", OneHotEncoder())
])

preprocessing = ColumnTransformer([
    ("numeric", num_pipeline, class_name),
    ("categorical", cat_pipeline, class_name2)
])

X_train,X_test, y_train, y_test = train_test_split(X, y, test_size=.4, random_state= 101)

X_train = preprocessing.fit_transform(X_train)
X_test = preprocessing.transform(X_test)

In [89]:
df

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,MALE
...,...,...,...,...,...,...,...
338,Gentoo,Biscoe,47.2,13.7,214.0,4925.0,FEMALE
340,Gentoo,Biscoe,46.8,14.3,215.0,4850.0,FEMALE
341,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,MALE
342,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,FEMALE


In [90]:
X_train.shape

(199, 11)

In [91]:
tf.random.set_seed(42)
input_layer = tf.keras.layers.Input(shape=[11])
flatten_layer = tf.keras.layers.Flatten() (input_layer)
dense_layer_1 = tf.keras.layers.Dense(400, 'relu')(flatten_layer)
dense_layer_2 = tf.keras.layers.Dense(200, 'relu')(dense_layer_1)
dense_layer_3 = tf.keras.layers.Dense(100, 'relu')(dense_layer_2)
dense_layer_4 = tf.keras.layers.Dense(100, 'relu')(dense_layer_3)
output_layer = tf.keras.layers.Dense(1)(dense_layer_4)

model = tf.keras.Model(inputs=[input_layer], outputs=[output_layer])
model.summary()

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 11)]              0         
                                                                 
 flatten_8 (Flatten)         (None, 11)                0         
                                                                 
 dense_32 (Dense)            (None, 400)               4800      
                                                                 
 dense_33 (Dense)            (None, 200)               80200     
                                                                 
 dense_34 (Dense)            (None, 100)               20100     
                                                                 
 dense_35 (Dense)            (None, 100)               10100     
                                                                 
 dense_36 (Dense)            (None, 1)                 101 

In [92]:
model.compile(loss= "mse", optimizer="adam", metrics=['RootMeanSquaredError'])

In [93]:
penguin_history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs= 150)

Epoch 1/150
7/7 [==============================] - 2s 36ms/step - loss: 18760214.0000 - root_mean_squared_error: 4331.3062 - val_loss: 17712360.0000 - val_root_mean_squared_error: 4208.6055
Epoch 2/150
7/7 [==============================] - 0s 12ms/step - loss: 18738524.0000 - root_mean_squared_error: 4328.8018 - val_loss: 17667828.0000 - val_root_mean_squared_error: 4203.3115
Epoch 3/150
7/7 [==============================] - 0s 12ms/step - loss: 18662088.0000 - root_mean_squared_error: 4319.9639 - val_loss: 17524672.0000 - val_root_mean_squared_error: 4186.2480
Epoch 4/150
7/7 [==============================] - 0s 9ms/step - loss: 18431674.0000 - root_mean_squared_error: 4293.2124 - val_loss: 17121070.0000 - val_root_mean_squared_error: 4137.7612
Epoch 5/150
7/7 [==============================] - 0s 12ms/step - loss: 17815450.0000 - root_mean_squared_error: 4220.8354 - val_loss: 16117800.0000 - val_root_mean_squared_error: 4014.6980
Epoch 6/150
7/7 [==============================] - 

In [94]:
metrics.mean_squared_error(y_test, model.predict(X_test), squared = False)

5/5 [==============================] - 0s 3ms/step


298.13564652307576